#setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
import lightgbm as lgbm
import numpy as np
import os
import random
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgbm

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import yfinance as yf
import pandas as pd
from datetime import date
import pandas_datareader as pdr
#Importing basic libraries.
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
import seaborn as sns
import os
import time
import warnings
warnings.filterwarnings("ignore")

#Pandas setting for better visualization of dataframe.
pd.options.display.float_format='{:.2f}'.format
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format

!pip install scikit-optimize

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.0 MB/s eta 0:00:00


In [4]:
!pip install optuna
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.9 MB/s eta 0:00:00


In [2]:
import numpy as np
# Set display options to show all columns without truncation
pd.set_option('display.max_columns', None)
folder_path = '/content/drive/MyDrive/house/'
dfOriginal  = pd.read_csv(folder_path + 'train.csv')
dfTest=df  = pd.read_csv(folder_path + 'test.csv')
dfTrain=dfOriginal.copy()
dfTest.head()

df  = pd.read_csv(folder_path + 'test.csv')

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import numpy as np



from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.model_selection import TimeSeriesSplit

class ModelSelectionPipeline:
    def __init__(self, model, model_params, n_jobs=-1, n_trials=15):
        self.model = model
        self.model_params = model_params
        self.n_jobs = n_jobs
        self.best_model = None
        self.best_predictions = None
        self.n_trials = n_trials

    def fit_and_predict(self, X_train, y_train, X_test):
        # Apply Standard scaling to the features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Use TimeSeriesSplit for cross-validation
        tscv = TimeSeriesSplit(n_splits=12)

        print(f"Evaluating model: {self.model.__class__.__name__}")
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: self.objective(trial, X_train_scaled, y_train, tscv), n_trials=self.n_trials)
        print(f"Best trial for model {self.model.__class__.__name__}: score {study.best_value}, params {study.best_params}")

        # Wrap the model with RFE for feature selection
        rfe_model = RFE(self.model)
        self.best_model = rfe_model.set_params(**study.best_params)
        self.best_model.fit(X_train_scaled, y_train)
        self.best_predictions = self.best_model.predict(X_test_scaled)

    def objective(self, trial, X, y, cv):
        params = self.model.get_params(deep=True)
        for param, bounds in self.model_params.items():
            if isinstance(bounds[0], int):
                params[param] = trial.suggest_int(param, min(bounds), max(bounds))
            elif isinstance(bounds[0], float):
                params[param] = trial.suggest_float(param, min(bounds), max(bounds))
            else:
                params[param] = trial.suggest_categorical(param, bounds)
        self.model.set_params(**params)

        # Wrap the model with RFE for feature selection
        rfe_model = RFE(self.model)

        scores = cross_val_score(rfe_model, X, y, cv=cv, scoring='neg_mean_absolute_error', n_jobs=self.n_jobs)

        return -scores.mean()

# Rest of the code remains unchanged


def best_preds_one(models, param_grids, X_train_combined, X_test_combined, y, n_trials=3):
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_combined, y, test_size=0.2, random_state=42)

    test_predictions = []
    last_predictions = None

    for model, param_grid in zip(models, param_grids):
        pipeline = ModelSelectionPipeline(model, param_grid, n_trials=n_trials)
        pipeline.fit_and_predict(X_train, y_train, X_test_combined)

        # Apply the same MinMax scaling used during training
        scaler = MinMaxScaler()
        X_val_scaled = scaler.fit_transform(X_val)

        pipeline_mse = mean_squared_error(y_val, pipeline.best_model.predict(X_val_scaled))
        print(f"Model mean squared error: {pipeline_mse}")

        test_predictions.append(pipeline.best_model.predict(X_test_combined))
        print('------------------------------------------------------------------------')

        last_predictions = pipeline.best_model.predict(X_test_combined)

    return test_predictions, last_predictions




In [6]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [8]:
dfTrain

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.00,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.00,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.00,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.00,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.00,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.00,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.00,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.00,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.00,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.00,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.00,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.00,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.00,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.00,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.00,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.00,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.00,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.00,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.00,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.00,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.00,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.00,TA,TA,CBlock,TA,TA,Mn,

#funcers


In [ ]:

def get_first_non_nan_sorted(dataset):
    total_rows = len(dataset)

    # Initialize a list to store the first non-NaN values and their indices
    first_non_nan_values = []

    for column in dataset.columns:
        # Get the column values excluding NaNs
        column_values = dataset[column].values
        non_nan_indices = np.where(~np.isnan(column_values))[0]

        # Check if there are any non-NaN values
        if len(non_nan_indices) > 0:
            # Retrieve the first non-NaN value and its index
            first_non_nan_value = column_values[non_nan_indices[0]]
            index_type = type(dataset.index[0])
            first_non_nan_index = index_type(dataset.index[non_nan_indices[0]])

            # Add the first non-NaN value and its index to the list
            first_non_nan_values.append((column, first_non_nan_index, first_non_nan_value))

    # Sort the first non-NaN values based on the sorted indices
    sorted_first_non_nan_values = sorted(first_non_nan_values, key=lambda x: x[1])

    return sorted_first_non_nan_values



def plot_columns(dataframe, n=10, figsize=(20, 16)):
    # Apply min-max scaling to the selected columns
    scaler = MinMaxScaler()
    scaled_columns = pd.DataFrame(scaler.fit_transform(dataframe),
                                  columns=dataframe.columns,
                                  index=dataframe.index)

    # Number of graphs
    num_graphs = len(dataframe.columns) // n if len(dataframe.columns) % n == 0 else len(dataframe.columns) // n + 1


    for k in range(num_graphs):
        start = k * n
        end = (k + 1) * n if (k + 1) * n < len(dataframe.columns) else len(dataframe.columns)

        fig, ax = plt.subplots(figsize=figsize)

        # Plot each column as a line
        for column in dataframe.columns[start:end]:
            ax.plot(scaled_columns.index, scaled_columns[column], label=column)

        # Set the x-axis label
        ax.set_xlabel('Index')

        # Set the y-axis label
        ax.set_ylabel('Scaled Value')

        # Set the title
        ax.set_title(f'Columns Plot {k+1}')

        # Add a legend
        ax.legend()

        # Display the plot
        plt.show()
def print_nan_zero_percentage(dataset):
    total_rows = len(dataset)

    # Calculate percentage of NaN and 0 values for each column
    nan_zero_percentage = []
    for column in dataset.columns:
        nan_percentage = dataset[column].isna().sum() / total_rows * 100
        zero_percentage = (dataset[column] == 0).sum() / total_rows * 100

        nan_zero_percentage.append((column, nan_percentage, zero_percentage))

    # Sort the percentages in descending order based on NaN percentage
    sorted_nan_zero_percentage = sorted(nan_zero_percentage, key=lambda x: x[1], reverse=True)

    # Print the sorted percentages
    for column, nan_percentage, zero_percentage in sorted_nan_zero_percentage:
        print(f"Column: {column}")
        print(f"NaN Percentage: {nan_percentage:.2f}%")
        print(f"Zero Percentage: {zero_percentage:.2f}%")
        print("------------------------")

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

def plot_yearly(df, start, end, start_date, end_date):
    # Plot each column
    for column in df.columns[start:end+1]:
        # Create a new figure for this column
        fig, ax = plt.subplots(figsize=(12, 6))

        # For each year
        for year in range(pd.to_datetime(start_date).year, pd.to_datetime(end_date).year):
            # Get data for this year
            start_of_year = max(pd.to_datetime(f"{year}-01-01"), pd.to_datetime(start_date))
            end_of_year = min(pd.to_datetime(f"{year+1}-01-01") - pd.Timedelta(days=1), pd.to_datetime(end_date))
            yearly_data = df[(df.index >= start_of_year) & (df.index <= end_of_year)][column]

            # Apply min-max scaling to the selected columns
            scaler = MinMaxScaler()
            yearly_data = pd.Series(scaler.fit_transform(yearly_data.values.reshape(-1, 1)).flatten(),
                                    index=yearly_data.index)

            # Create a continuous date index and interpolate missing data
            all_days = pd.date_range(start=start_of_year, end=end_of_year, freq='D')
            yearly_data = yearly_data.reindex(all_days).interpolate(method='time')

            # Plot data
            ax.plot(yearly_data.index.dayofyear, yearly_data.values, label=year)

        # Set labels and title
        ax.set_xlabel('Day of Year')
        ax.set_ylabel('Scaled Value')
        ax.set_title(f'{column} - Yearly Comparison')

        # Simplify x-axis to 12 ticks
        ax.set_xticks(np.linspace(1, 365, num=12))
        ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

        # Show legend
        ax.legend()

        # Show the plot
        plt.show()

import matplotlib.patches as mpatches

def visualize_data(df, figsize=(15, 10)):
    n = 10  # Number of columns per plot
    num_plots = np.ceil(df.shape[1] / n).astype(int)  # Calculate the number of plots needed

    for i in range(num_plots):
        fig, ax = plt.subplots(figsize=figsize)  # Set the figure size

        # Select the columns for this plot
        columns = df.columns[i * n: (i + 1) * n]

        for col_num, col_name in enumerate(columns):
            col_data = df[col_name]

            # Iterate over rows and stack them in the bar plot
            for row_num, value in enumerate(col_data):
                if pd.isna(value):
                    color = 'red'
                elif value == 0:
                    color = 'blue'
                else:
                    color = 'green'
                ax.bar(col_num, 1, bottom=row_num, color=color)

        # Set Y-axis labels as row indices for selected rows only
        selected_ticks = range(0, len(df.index), len(df.index)//10)  # Select every 10% of data for labeling
        ax.set_yticks(selected_ticks)
        ax.set_yticklabels(df.index[selected_ticks])

        # Set X-axis labels as column names
        ax.set_xticks(range(len(columns)))
        ax.set_xticklabels(columns)

        # Create legend
        red_patch = mpatches.Patch(color='red', label='NaN')
        blue_patch = mpatches.Patch(color='blue', label='Zero')
        green_patch = mpatches.Patch(color='green', label='Non-Zero')
        ax.legend(handles=[red_patch, blue_patch, green_patch])

        plt.show()

In [ ]:
from google.colab import files

def create_and_download_submission(predictions, filename='submission.csv'):
    # Read the CSV file that contains the ID of each test sample
    dfid = pd.read_csv(folder_path + 'testR.csv')

    # Create a dataframe for the submission
    submission = pd.DataFrame({
        "ID_LAT_LON_YEAR_WEEK": dfid["ID_LAT_LON_YEAR_WEEK"],
        "emission": predictions
    })

    # Save the submission dataframe to a CSV file
    submission.to_csv(filename, index=False)

    # Download the CSV file
    files.download(filename)

In [ ]:
def print_column_summary(df):
    for col in df.columns:
        unique_values = df[col].nunique()
        top_values = df[col].value_counts(dropna=False).head(5).to_dict()
        top_values_str = "///".join([f'[{str(k)} = {v}]' for k, v in top_values.items()])

        print(f'Column: {col} - number of unique values = {unique_values}/ Top 5 {top_values_str}')


In [ ]:
import pandas as pd

def fill_nans_with_previous(df):
    for column in df.columns:
        # Fill NaNs with the previous non-NaN values
        df[column].fillna(method='ffill', inplace=True)

        # If the column starts with NaN values, fill with the next non-NaN value
        if pd.isna(df[column].iloc[0]):
            df[column].fillna(method='bfill', inplace=True)

    return df



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def eda_function(df, target_column,
                 figsize_heatmap=(12, 8),
                 figsize_corr=(20, 20),
                 figsize_hist=(20, 20),
                 bins_hist=20,
                 figsize_scatter=(20, 20),
                 selected_features=None,
                 num_hist_per_fig=9,
                 num_scatter_per_fig=6,
                 num_features_per_heatmap=10,
                 num_features_per_pairplot=6):


    print('*'*180)
    print_nan_zero_percentage(df)
    print('*'*180)
    # Summary statistics
    print(df.describe())

    print('*'*180)
    print('*'*180)
    print()

    # Histograms for numerical features
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for i in range(0, len(numerical_cols), num_hist_per_fig):
        df[numerical_cols[i:i + num_hist_per_fig]].hist(figsize=figsize_hist, bins=bins_hist)
        plt.show()
        print('-'*180)




    print('*'*180)
    print('*'*180)
    print()

    # Scatter plots to observe relationships
    for i in range(0, len(numerical_cols) - 1, num_scatter_per_fig):
        fig, axes = plt.subplots(1, num_scatter_per_fig, figsize=figsize_scatter, sharey=True)
        for j in range(num_scatter_per_fig):
            if i + j < len(numerical_cols) - 1:
                column = numerical_cols[i + j]
                axes[j].scatter(df[column], df[target_column])
                axes[j].set_xlabel(column)
                axes[j].set_ylabel(target_column)
                axes[j].set_title(f"{column} vs {target_column}")
        plt.show()
        print('-'*180)

    print('*'*180)
    print('*'*180)
    print()

    # Pairplot for selected features (avoiding all for heavy computational load)

    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.drop(target_column)
    for i in range(0, len(numerical_columns), num_features_per_pairplot):
        selected_features = numerical_columns[i:i+num_features_per_pairplot].to_list()
        selected_features.append(target_column)
        sns.pairplot(df[selected_features], y_vars=target_column, x_vars=selected_features)
        plt.show()

def analyze_nans(df):
    results = {}
    for col in df.columns:
        # Count consecutive NaNs
        max_consecutive_nans = 0
        current_consecutive_nans = 0
        # Count total NaNs for average calculation
        total_nans = 0

        for value in df[col]:
            if pd.isna(value):
                current_consecutive_nans += 1
                total_nans += 1
            else:
                if current_consecutive_nans > max_consecutive_nans:
                    max_consecutive_nans = current_consecutive_nans
                current_consecutive_nans = 0

        if current_consecutive_nans > max_consecutive_nans:
            max_consecutive_nans = current_consecutive_nans

        # Calculate average number of NaNs per 365 elements
        average_nans_per_365 = total_nans / (len(df[col]) / 365) if len(df[col]) > 0 else 0

        results[col] = {
            'largest_number_of_nans_in_a_row': max_consecutive_nans,
            'average_number_of_nans_per_365_elements': average_nans_per_365
        }

    return results



In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def plot_top_correlations(df, target_column, num_features_per_heatmap):
    # Drop columns with over 70% NaN values
    df = df.dropna(thresh=len(df) * 0.2, axis=1)

    # Drop remaining NaN values
    df = df.dropna()

    # Compute the correlation matrix
    corrmat = df.corr()

    # Get the correlations with the target column, sorted by absolute value
    sorted_correlations = corrmat[target_column].apply(abs).sort_values(ascending=False)
    for i in range(1, len(sorted_correlations), num_features_per_heatmap):
        # Get the next set of correlated features including the target column
        cols = [target_column] + sorted_correlations.index[i:i + num_features_per_heatmap].tolist()
        cm = np.corrcoef(df[cols].values.T)
        sns.set(font_scale=0.8) # Adjusting the font scale for smaller labels
        plt.figure(figsize=(20,20)) # Making the figure size larger
        hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.3f', annot_kws={'size': 8}, # Smaller annotation size
                        yticklabels=cols, xticklabels=cols, cmap='coolwarm')
        plt.title(f'Heatmap of Top {num_features_per_heatmap} Correlations with {target_column}')
        plt.show()
        print("\n" * 3 + "=" * 180 + "\n" * 3)








#heheheheh

In [167]:
import numpy as np
# Set display options to show all columns without truncation
pd.set_option('display.max_columns', None)
folder_path = '/content/drive/MyDrive/house/'
dfOriginal  = pd.read_csv(folder_path + 'train.csv')
dfTest=df  = pd.read_csv(folder_path + 'test.csv')
dfTrain=dfOriginal.copy()
dfTest.head()

df  = pd.read_csv(folder_path + 'test.csv')
dfTest

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.00,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.00,TA,TA,CBlock,TA,TA,No,Rec,468.00,LwQ,144.00,270.00,882.00,GasA,TA,Y,SBrkr,896,0,0,896,0.00,0.00,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.00,Unf,1.00,730.00,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.00,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.00,TA,TA,CBlock,TA,TA,No,ALQ,923.00,Unf,0.00,406.00,1329.00,GasA,TA,Y,SBrkr,1329,0,0,1329,0.00,0.00,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.00,Unf,1.00,312.00,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.00,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.00,TA,TA,PConc,Gd,TA,No,GLQ,791.00,Unf,0.00,137.00,928.00,GasA,Gd,Y,SBrkr,928,701,0,1629,0.00,0.00,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.00,Fin,2.00,482.00,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.00,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.00,TA,TA,PConc,TA,TA,No,GLQ,602.00,Unf,0.00,324.00,926.00,GasA,Ex,Y,SBrkr,926,678,0,1604,0.00,0.00,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.00,Fin,2.00,470.00,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.00,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.00,Gd,TA,PConc,Gd,TA,No,ALQ,263.00,Unf,0.00,1017.00,1280.00,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.00,0.00,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.00,RFn,2.00,506.00,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.00,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.00,TA,TA,CBlock,TA,TA,No,Unf,0.00,Unf,0.00,546.00,546.00,GasA,Gd,Y,SBrkr,546,546,0,1092,0.00,0.00,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.00,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.00,TA,TA,CBlock,TA,TA,No,Rec,252.00,Unf,0.00,294.00,546.00,GasA,TA,Y,SBrkr,546,546,0,1092,0.00,0.00,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.00,Unf,1.00,286.00,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.00,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.00,TA,TA,CBlock,TA,TA,No,ALQ,1224.00,Unf,0.00,0.00,1224.00,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.00,0.00,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.00,Unf,2.00,576.00,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.00,10441,P

In [169]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Concatenating the train and test dataframes (assuming you have them as dfTrain and dfTest)
df_combined = pd.concat([dfTrain, dfTest], keys=['train', 'test'])



def transform_dataframe(df):
    # Creating new columns for "YearsSinceBuilt" and "YearsSinceRemodel"
    df['YearsSinceBuilt'] = df['YrSold'] - df['YearBuilt']
    df['YearsSinceRemodel'] = df['YrSold'] - df['YearRemodAdd']
    df.drop(['SalePrice'], axis=1, inplace=True)

    #df = df.fillna(df.median())
    df.drop(['Id', 'YearBuilt', 'YearRemodAdd'], axis=1, inplace=True)

    # Sum all the surface area columns and create a new column 'TotalSurfaceArea'
    surface_area_columns = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'WoodDeckSF', 'OpenPorchSF']
    surface_area_house=['TotalBsmtSF',  '1stFlrSF', '2ndFlrSF']

    df['TotalSurfaceArea'] = df[surface_area_house].sum(axis=1)
    df['SurfaceArea'] = df[surface_area_columns].sum(axis=1)

    #df.drop(['BsmtFinSF1','BsmtFinSF2','TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'WoodDeckSF', 'OpenPorchSF'], axis=1, inplace=True)
    # Iterating through the columns to apply encoding
    for column in df.columns:
        if df[column].dtype == 'object':
            unique_values = df[column].nunique()

            # Binary encoding if 2 unique values
            if unique_values == 2:
                df[column] = (df[column] == df[column].mode()[0]).astype(int)

            # One-hot encoding (dummy encoding) if 3-4 unique values
            elif unique_values in [3, 4]:
                dummies = pd.get_dummies(df[column], prefix=column, drop_first=True)
                df = pd.concat([df, dummies], axis=1)
                df.drop(column, axis=1, inplace=True)

            # Label encoding for other non-numeric columns
            else:
                le = LabelEncoder()
                df[column] = le.fit_transform(df[column].fillna('Missing'))

    df = df.fillna(df.median())

    return df

# Applying the transformation
df_combined_transformed = transform_dataframe(df_combined)

# Splitting back into train and test dataframes
dfTrain = df_combined_transformed.loc['train']
dfTest = df_combined_transformed.loc['test']





In [170]:
dfTest

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterCond,Foundation,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearsSinceBuilt,YearsSinceRemodel,TotalSurfaceArea,SurfaceArea,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,20,3,80.00,11622,1,0,1,4,12,1,2,0,2,5,6,1,1,13,14,0.00,4,1,5,468.00,3,144.00,270.00,882.00,1,4,1,5,896,0,0,896,0.00,0.00,1,0,2,1,5,7,0,3,1,1961.00,1.00,730.00,5,5,140,0,0,0,120,0,0,6,2010,9,4,49,49,1778.00,1918.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0
1,20,4,81.00,14267,1,0,1,0,12,2,2,0,2,6,6,3,1,14,15,108.00,4,1,0,923.00,6,0.00,406.00,1329.00,1,4,1,5,1329,0,0,1329,0.00,0.00,1,1,3,1,6,7,0,3,1,1958.00,1.00,312.00,5,5,393,36,0,0,0,0,12500,6,2010,9,4,52,52,2658.00,3087.00,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
2,60,4,74.00,13830,1,0,1,4,8,2,2,0,5,5,5,1,1,13,14,0.00,4,2,2,791.00,6,0.00,137.00,928.00,1,2,1,5,928,701,0,1629,0.00,0.00,2,1,3,1,6,7,1,5,1,1997.00,2.00,482.00,5,5,212,34,0,0,0,0,0,3,2010,9,4,13,12,2557.00,2803.00,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,60,4,78.00,9978,1,0,1,4,8,2,2,0,5,6,6,1,1,13,14,20.00,4,2,2,602.00,6,0.00,324.00,926.00,1,0,1,5,926,678,0,1604,0.00,0.00,2,1,3,1,7,7,1,2,1,1998.00,2.00,470.00,5,5,360,36,0,0,0,0,0,6,2010,9,4,12,12,2530.00,2926.00,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,120,4,43.00,5005,1,0,1,4,22,2,2,4,2,8,5,1,1,6,6,0.00,4,2,0,263.00,6,0.00,1017.00,1280.00,1,0,1,5,1280,0,0,1280,0.00,0.00,2,0,2,1,5,7,0,3,1,1992.00,2.00,506.00,5,5,0,82,0,0,144,0,0,1,2010,9,4,18,18,2560.00,2642.00,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,5,21.00,1936,1,0,1,4,10,2,2,3,5,4,7,1,1,5,5,0.00,4,1,6,0.00,6,0.00,546.00,546.00,1,2,1,5,546,546,0,1092,0.00,0.00,1,1,3,1,5,7,0,3,6,1979.00,0.00,0.00,3,3,0,0,0,0,0,0,0,6,2006,9,4,36,36,1638.00,1638.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1455,160,5,21.00,1894,1,0,1,4,10,2,2,4,5,4,5,1,1,5,5,0.00,4,1,5,252.00,6,0.00,294.00,546.00,1,4,1,5,546,546,0,1092,0.00,0.00,1,1,3,1,6,7,0,3,4,1970.00,1.00,286.00,5,5,0,24,0,0,0,0,0,4,2006,9,0,36,36,1638.00,1662.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0
1456,20,4,160.00,20000,1,0,1,4,11,2,2,0,2,5,7,1,1,13,14,0.00,4,1,0,1224.00,6,0.00,0.00,1224.00,1,0,1,5,1224,0,0,1224,1.00,0.00,1,0,4,1,7,7,1,5,5,1960.00,2.00,576.00,5,5,474,0,0,0,0,0,0,9,2006,9,0,46,10,2448.00,2922.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0

In [138]:
dfTrain

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterCond,Foundation,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearsSinceBuilt,YearsSinceRemodel,TotalSurfaceArea,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,60,4,65.00,8450,1,0,1,4,5,2,2,0,5,7,5,1,1,13,14,196.00,4,2,2,706.00,6,0.00,150.00,856.00,1,0,1,5,856,854,0,1710,1.00,0.00,2,1,3,1,8,7,0,3,1,2003.00,2.00,548.00,5,5,0,61,0,0,0,0,0,2,2008,9,4,5,5,3483.00,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
1,20,4,80.00,9600,1,0,1,2,24,1,2,0,2,6,8,1,1,8,8,0.00,4,1,0,978.00,6,0.00,284.00,1262.00,1,0,1,5,1262,0,0,1262,0.00,1.00,2,0,3,1,6,7,1,5,1,1976.00,2.00,460.00,5,5,298,0,0,0,0,0,0,5,2007,9,4,31,31,4084.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
2,60,4,68.00,11250,1,0,1,4,5,2,2,0,5,7,5,1,1,13,14,162.00,4,2,2,486.00,6,0.00,434.00,920.00,1,0,1,5,920,866,0,1786,1.00,0.00,2,1,3,1,6,7,1,5,1,2001.00,2.00,608.00,5,5,0,42,0,0,0,0,0,9,2008,9,4,7,6,3668.00,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
3,70,4,60.00,9550,1,0,1,0,6,2,2,0,5,7,5,1,1,14,16,0.00,4,0,0,216.00,6,0.00,540.00,756.00,1,2,1,5,961,756,0,1717,1.00,0.00,1,0,3,1,7,7,1,2,5,1998.00,3.00,642.00,5,5,0,35,272,0,0,0,0,2,2006,9,0,91,36,3536.00,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
4,60,4,84.00,14260,1,0,1,2,15,2,2,0,5,8,5,1,1,13,14,350.00,4,2,2,655.00,6,0.00,490.00,1145.00,1,0,1,5,1145,1053,0,2198,1.00,0.00,2,1,4,1,9,7,1,5,1,2000.00,3.00,836.00,5,5,192,84,0,0,0,0,0,12,2008,9,4,8,8,4764.00,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,4,62.00,7917,1,0,1,4,8,2,2,0,5,6,5,1,1,13,14,0.00,4,2,6,0.00,6,0.00,953.00,953.00,1,0,1,5,953,694,0,1647,0.00,0.00,2,1,3,1,7,7,1,5,1,1999.00,2.00,460.00,5,5,0,40,0,0,0,0,0,8,2007,9,4,8,7,3593.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
1456,20,4,85.00,13175,1,0,1,4,14,2,2,0,2,6,6,1,1,10,11,119.00,4,1,0,790.00,5,163.00,589.00,1542.00,1,4,1,5,2073,0,0,2073,1.00,0.00,2,0,3,1,7,2,2,5,1,1978.00,2.00,500.00,5,5,349,0,0,0,0,0,0,2,2010,9,4,32,22,5506.00,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0
1457,70,4,66.00,9042,1,0,1,4,6,2,2,0,5,7,9,1,1,5,5,0.00,2,4,2,275.00,6,0.00,877.00,1152.00,1,0,1,5,1188,1152,0,2340,0.00,0.00,2,0,4,1,9,7,2,2,1,1941.00,1.00,252.00,5,5,0,60,0,0,0,0,2500,5,2010,9,4,69,4,4704.00,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
1458,20,4,68.00,9717,1,0,1,4,12,2,2,0,2,5,6,3,1,8,8

In [41]:
dfOriginal

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.00,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.00,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.00,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.00,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.00,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.00,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.00,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.00,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.00,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.00,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.00,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.00,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.00,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.00,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.00,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.00,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.00,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.00,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.00,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.00,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.00,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.00,TA,TA,CBlock,TA,TA,Mn,

In [40]:
dfTrain['SalePrice']=dfOriginal['SalePrice']
dfTrain

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterCond,Foundation,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,YearsSinceBuilt,YearsSinceRemodel,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,60,4,65.00,8450,1,0,1,4,5,2,2,0,5,7,5,2003,2003,1,1,13,14,196.00,4,2,2,706.00,6,0.00,150.00,856.00,1,0,1,5,856,854,0,1710,1.00,0.00,2,1,3,1,8,7,0,3,1,2003.00,2.00,548.00,5,5,0,61,0,0,0,0,0,2,2008,9,4,208500,5,5,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
1,20,4,80.00,9600,1,0,1,2,24,1,2,0,2,6,8,1976,1976,1,1,8,8,0.00,4,1,0,978.00,6,0.00,284.00,1262.00,1,0,1,5,1262,0,0,1262,0.00,1.00,2,0,3,1,6,7,1,5,1,1976.00,2.00,460.00,5,5,298,0,0,0,0,0,0,5,2007,9,4,181500,31,31,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
2,60,4,68.00,11250,1,0,1,4,5,2,2,0,5,7,5,2001,2002,1,1,13,14,162.00,4,2,2,486.00,6,0.00,434.00,920.00,1,0,1,5,920,866,0,1786,1.00,0.00,2,1,3,1,6,7,1,5,1,2001.00,2.00,608.00,5,5,0,42,0,0,0,0,0,9,2008,9,4,223500,7,6,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
3,70,4,60.00,9550,1,0,1,0,6,2,2,0,5,7,5,1915,1970,1,1,14,16,0.00,4,0,0,216.00,6,0.00,540.00,756.00,1,2,1,5,961,756,0,1717,1.00,0.00,1,0,3,1,7,7,1,2,5,1998.00,3.00,642.00,5,5,0,35,272,0,0,0,0,2,2006,9,0,140000,91,36,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
4,60,4,84.00,14260,1,0,1,2,15,2,2,0,5,8,5,2000,2000,1,1,13,14,350.00,4,2,2,655.00,6,0.00,490.00,1145.00,1,0,1,5,1145,1053,0,2198,1.00,0.00,2,1,4,1,9,7,1,5,1,2000.00,3.00,836.00,5,5,192,84,0,0,0,0,0,12,2008,9,4,250000,8,8,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,4,62.00,7917,1,0,1,4,8,2,2,0,5,6,5,1999,2000,1,1,13,14,0.00,4,2,6,0.00,6,0.00,953.00,953.00,1,0,1,5,953,694,0,1647,0.00,0.00,2,1,3,1,7,7,1,5,1,1999.00,2.00,460.00,5,5,0,40,0,0,0,0,0,8,2007,9,4,175000,8,7,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
1456,20,4,85.00,13175,1,0,1,4,14,2,2,0,2,6,6,1978,1988,1,1,10,11,119.00,4,1,0,790.00,5,163.00,589.00,1542.00,1,4,1,5,2073,0,0,2073,1.00,0.00,2,0,3,1,7,2,2,5,1,1978.00,2.00,500.00,5,5,349,0,0,0,0,0,0,2,2010,9,4,210000,32,22,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0
1457,70,4,66.00,9042,1,0,1,4,6,2,2,0,5,7,9,1941,2006,1,1,5,5,0.00,2,4,2,275.00,6,0.00,877.00,1152.00,1,0,1,5,1188,1152,0,2340,0.00,0.00,2,0,4,1,9,7,2,2,1,1941.00,1.00,252.00,5,5,0,60,0,0,0,0,2500,5,2010,9,4,266500,69,4,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0

In [66]:
dfTest

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterCond,Foundation,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,YearsSinceBuilt,YearsSinceRemodel,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,20,3,80.00,11622,1,0,1,4,12,1,2,0,2,5,6,1961,1961,1,1,13,14,0.00,4,1,5,468.00,3,144.00,270.00,882.00,1,4,1,5,896,0,0,896,0.00,0.00,1,0,2,1,5,7,0,3,1,1961.00,1.00,730.00,5,5,140,0,0,0,120,0,0,6,2010,9,4,0.00,49,49,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0
1,20,4,81.00,14267,1,0,1,0,12,2,2,0,2,6,6,1958,1958,3,1,14,15,108.00,4,1,0,923.00,6,0.00,406.00,1329.00,1,4,1,5,1329,0,0,1329,0.00,0.00,1,1,3,1,6,7,0,3,1,1958.00,1.00,312.00,5,5,393,36,0,0,0,0,12500,6,2010,9,4,0.00,52,52,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
2,60,4,74.00,13830,1,0,1,4,8,2,2,0,5,5,5,1997,1998,1,1,13,14,0.00,4,2,2,791.00,6,0.00,137.00,928.00,1,2,1,5,928,701,0,1629,0.00,0.00,2,1,3,1,6,7,1,5,1,1997.00,2.00,482.00,5,5,212,34,0,0,0,0,0,3,2010,9,4,0.00,13,12,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
3,60,4,78.00,9978,1,0,1,4,8,2,2,0,5,6,6,1998,1998,1,1,13,14,20.00,4,2,2,602.00,6,0.00,324.00,926.00,1,0,1,5,926,678,0,1604,0.00,0.00,2,1,3,1,7,7,1,2,1,1998.00,2.00,470.00,5,5,360,36,0,0,0,0,0,6,2010,9,4,0.00,12,12,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,120,4,43.00,5005,1,0,1,4,22,2,2,4,2,8,5,1992,1992,1,1,6,6,0.00,4,2,0,263.00,6,0.00,1017.00,1280.00,1,0,1,5,1280,0,0,1280,0.00,0.00,2,0,2,1,5,7,0,3,1,1992.00,2.00,506.00,5,5,0,82,0,0,144,0,0,1,2010,9,4,0.00,18,18,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,5,21.00,1936,1,0,1,4,10,2,2,3,5,4,7,1970,1970,1,1,5,5,0.00,4,1,6,0.00,6,0.00,546.00,546.00,1,2,1,5,546,546,0,1092,0.00,0.00,1,1,3,1,5,7,0,3,6,0.00,0.00,0.00,3,3,0,0,0,0,0,0,0,6,2006,9,4,0.00,36,36,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1455,160,5,21.00,1894,1,0,1,4,10,2,2,4,5,4,5,1970,1970,1,1,5,5,0.00,4,1,5,252.00,6,0.00,294.00,546.00,1,4,1,5,546,546,0,1092,0.00,0.00,1,1,3,1,6,7,0,3,4,1970.00,1.00,286.00,5,5,0,24,0,0,0,0,0,4,2006,9,0,0.00,36,36,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0
1456,20,4,160.00,20000,1,0,1,4,11,2,2,0,2,5,7,1960,1996,1,1,13,14,0.00,4,1,0,1224.00,6,0.00,0.00,1224.00,1,0,1,5,1224,0,0,1224,1.00,0.00,1,0,4,1,7,7,1,5,5,1960.00,2.00,576.00,5,5,474,0,0,0,0,0,0,9,2006,9,0,0.00,46,10,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

class ModelSelectionPipeline:
    def __init__(self, model, model_params, n_jobs=-1, n_trials=15):
        self.model = model
        self.model_params = model_params
        self.n_jobs = n_jobs
        self.best_model = None
        self.best_predictions = None
        self.n_trials=n_trials

    def fit_and_predict(self, X_train, y_train, X_test):



        print(f"Evaluating model: {self.model.__class__.__name__}")
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: self.objective(trial, X_train, y_train),  n_trials=self.n_trials)
        print(f"Best trial for model {self.model.__class__.__name__}: score {study.best_value}, params {study.best_params}")


        self.best_model = self.model.set_params(**study.best_params)
        self.best_model.fit(X_train, y_train)
        self.best_predictions = self.best_model.predict(X_test)

    def objective(self, trial, X, y):
        params = self.model.get_params(deep=True)  # Start with the model's default parameters
        for param, bounds in self.model_params.items():
            if isinstance(bounds[0], int):
                params[param] = trial.suggest_int(param, min(bounds), max(bounds))
            elif isinstance(bounds[0], float):
                params[param] = trial.suggest_float(param, min(bounds), max(bounds))
            else:
                params[param] = trial.suggest_categorical(param, bounds)
        self.model.set_params(**params)

        # Use MAE as the scoring metric
        scores = cross_val_score(self.model, X, y, cv=10, scoring='neg_mean_absolute_error', n_jobs=self.n_jobs)

        return -scores.mean()  # Minimize the negative mean absolute error

def best_preds_one(models, param_grids, X_train_combined, X_test_combined, y, n_trials=10):
    # Split the combined training data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_combined, y, test_size=0.2, random_state=42)

    test_predictions = []
    last_predictions = None

    for model, param_grid in zip(models, param_grids):
        pipeline = ModelSelectionPipeline(model, param_grid, n_trials=n_trials)
        pipeline.fit_and_predict(X_train, y_train, X_test_combined)

        pipeline_mse = mean_squared_error(y_val, pipeline.best_model.predict(X_val))
        print(f"Model mean squared error: {pipeline_mse}")



        test_predictions.append(pipeline.best_model.predict(X_test_combined))
        print('------------------------------------------------------------------------')

        last_predictions = pipeline.best_model.predict(X_test_combined)

    return test_predictions, last_predictions


In [74]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from math import sqrt
import optuna

class ModelSelectionPipeline:
    def __init__(self, model, model_params, n_jobs=-1, n_trials=15):
        self.model = model
        self.model_params = model_params
        self.n_jobs = n_jobs
        self.best_model = None
        self.best_predictions = None
        self.n_trials = n_trials

    def fit_and_predict(self, X_train, y_train, X_test):
        print(f"Evaluating model: {self.model.__class__.__name__}")
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: self.objective(trial, X_train, y_train), n_trials=self.n_trials)
        print(f"Best trial for model {self.model.__class__.__name__}: score {study.best_value}, params {study.best_params}")

        self.best_model = self.model.set_params(**study.best_params)
        self.best_model.fit(X_train, y_train)
        self.best_predictions = self.best_model.predict(X_test)

    def objective(self, trial, X, y):
        params = self.model.get_params(deep=True)
        for param, bounds in self.model_params.items():
            if isinstance(bounds[0], int):
                params[param] = trial.suggest_int(param, min(bounds), max(bounds))
            elif isinstance(bounds[0], float):
                params[param] = trial.suggest_float(param, min(bounds), max(bounds))
            else:
                params[param] = trial.suggest_categorical(param, bounds)
        self.model.set_params(**params)

        # Use RMSE as the scoring metric
        scores = cross_val_score(self.model, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=self.n_jobs)
        rmse_scores = [sqrt(-score) for score in scores]

        return sum(rmse_scores) / len(rmse_scores)  # Minimize the mean RMSE

def best_preds_one(models, param_grids, X_train_combined, X_test_combined, y, n_trials=10):
    X_train, X_val, y_train, y_val = train_test_split(X_train_combined, y, test_size=0.2, random_state=42)

    test_predictions = []
    last_predictions = None

    for model, param_grid in zip(models, param_grids):
        pipeline = ModelSelectionPipeline(model, param_grid, n_trials=n_trials)
        pipeline.fit_and_predict(X_train, y_train, X_test_combined)

        pipeline_mse = mean_squared_error(y_val, pipeline.best_model.predict(X_val))
        pipeline_rmse = sqrt(pipeline_mse)
        print(f"Model root mean squared error: {pipeline_rmse}")

        test_predictions.append(pipeline.best_model.predict(X_test_combined))
        print('------------------------------------------------------------------------')

        last_predictions = pipeline.best_model.predict(X_test_combined)

    return test_predictions, last_predictions


In [162]:
import numpy as np
import optuna
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split
from math import sqrt

class ModelSelectionPipeline:
    def __init__(self, model, model_params, n_jobs=-1, n_trials=15):
        self.model = model
        self.model_params = model_params
        self.n_jobs = n_jobs
        self.best_model = None
        self.best_predictions = None
        self.n_trials = n_trials

    def fit_and_predict(self, X_train, y_train, X_test):
        print(f"Evaluating model: {self.model.__class__.__name__}")
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: self.objective(trial, X_train, y_train), n_trials=self.n_trials)
        print(f"Best trial for model {self.model.__class__.__name__}: score {study.best_value}, params {study.best_params}")

        self.best_model = self.model.set_params(**study.best_params)
        self.best_model.fit(X_train, y_train)
        self.best_predictions = self.best_model.predict(X_test)

    def objective(self, trial, X, y):
        params = self.model.get_params(deep=True)
        for param, bounds in self.model_params.items():
            if isinstance(bounds[0], int):
                params[param] = trial.suggest_int(param, min(bounds), max(bounds))
            elif isinstance(bounds[0], float):
                params[param] = trial.suggest_float(param, min(bounds), max(bounds))
            else:
                params[param] = trial.suggest_categorical(param, bounds)
        self.model.set_params(**params)

        # Custom scoring function for RMSE between logarithms
        def log_rmse(y_true, y_pred):
            return sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

        log_rmse_scorer = make_scorer(log_rmse, greater_is_better=False)

        scores = cross_val_score(self.model, X, y, cv=10, scoring=log_rmse_scorer, n_jobs=self.n_jobs)

        return -scores.mean()  # Minimize the negative log RMSE

from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score

def best_preds_one(models, param_grids, X_train_combined, X_test_combined, y, n_trials=10):
    X_train, X_val, y_train, y_val = train_test_split(X_train_combined, y, test_size=0.2, random_state=42)

    test_predictions = []
    last_predictions = None

    for model, param_grid in zip(models, param_grids):
        pipeline = ModelSelectionPipeline(model, param_grid, n_trials=n_trials)
        pipeline.fit_and_predict(X_train, y_train, X_test_combined)

        y_pred_val = pipeline.best_model.predict(X_val)

        # Calculating various metrics
        mae = mean_absolute_error(y_val, y_pred_val)
        mse = mean_squared_error(y_val, y_pred_val)
        rmse = sqrt(mse)
        rmsle = sqrt(mean_squared_error(np.log1p(y_val), np.log1p(y_pred_val)))
        r2 = r2_score(y_val, y_pred_val)
        evs = explained_variance_score(y_val, y_pred_val)

        # Printing the metrics
        print(f"Model {model.__class__.__name__} Metrics:")
        print(f"Mean Absolute Error (MAE): {mae}")
        print(f"Mean Squared Error (MSE): {mse}")
        print(f"Root Mean Squared Error (RMSE): {rmse}")
        print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle}")
        print(f"R² score: {r2}")
        print(f"Explained Variance Score: {evs}")
        print('------------------------------------------------------------------------')

        test_predictions.append(pipeline.best_model.predict(X_test_combined))
        last_predictions = pipeline.best_model.predict(X_test_combined)

    return test_predictions, last_predictions



In [109]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Gradient Boosting Regressor
from sklearn.ensemble import GradientBoostingRegressor




# AdaBoost Regressor
from sklearn.ensemble import AdaBoostRegressor



# XGBoost Regressor
from xgboost import XGBRegressor



# Linear Regression
from sklearn.linear_model import LinearRegression

# Lasso Regression
# Lasso Regression
lasso = Lasso()
lasso_params = {
    'alpha': [0.01, 100],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'max_iter': [500, 10000],
    'tol': [0.00001, 0.01],
    'warm_start': [True, False],
    'positive': [True, False],
    'selection': ['cyclic', 'random']
}

# Ridge Regression
ridge = Ridge()
ridge_params = {
    'alpha': [0.01, 100],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'max_iter': [500, 2000],
    'tol': [0.00001, 0.01],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

# Random Forest Regressor
random_forest = RandomForestRegressor()
rf_params = {
    'n_estimators': [25, 100],
    'max_depth': [3, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 10],
    'max_features': ['sqrt'],
    #'bootstrap': [True, False],
    #'oob_score': [False, True],
    #'warm_start': [False, True]
}

# Gradient Boosting Regressor
gradient_boosting = GradientBoostingRegressor()
gbr_params = {
    'loss': ['ls', 'lad', 'huber', 'quantile'],
    'learning_rate': [0.001, 0.5],
    'n_estimators': [25, 100],
    'subsample': [0.1, 1.0],
    'warm_start': [False, True],
    'validation_fraction': [0.05, 0.5],
    'tol': [1e-5, 1e-3]
}

# AdaBoost Regressor
adaboost = AdaBoostRegressor()
ada_params = {
    'n_estimators': [25, 100],
    'learning_rate': [0.001, 0.5],
    'loss': ['linear', 'square', 'exponential']
}
xgboost = XGBRegressor()
xgboost_params ={
    'n_estimators': (50, 200),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.5),
    'gamma': (0, 0.5),
    'reg_lambda': (0.1, 1.0),
    'reg_alpha': (0.1, 1.0),
    'subsample': (0.5, 1.0)
    }



# Linear Regression
linear_regression = LinearRegression()
lr_params = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [False, True]
}


# Linear Regression
linear_regression = LinearRegression()
lr_params = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [False, True]
}

lgbm=LGBMRegressor()

lgbm_params = {
    'n_estimators': (200, 300),
    'learning_rate': (0.01, 0.5),
    'boosting_type': ['gbdt', 'dart', 'rf'],
    'num_leaves': (51, 200),
    'max_depth': (3, 10),
    'subsample_for_bin': (20000, 300000),
    'min_split_gain': (0.0, 1.0),
    'min_child_weight': (1e-3, 1e-1),
    'min_child_samples': (5, 50),
    'subsample': (0.5, 1.0),
    'subsample_freq': (0, 10),
    'colsample_bytree': (0.5, 1.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0)
}
xgboost = XGBRegressor()
xgboost_params ={
    'n_estimators': (50, 200),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.5),
    'gamma': (0, 0.5),
    'reg_lambda': (0.1, 1.0),
    'reg_alpha': (0.1, 1.0),
    'subsample': (0.5, 1.0)
    }



In [71]:
lgbm=LGBMRegressor()

lgbm_params = {
    'n_estimators': (216, 300),
    'learning_rate': (0.1086, 0.4925),
    'boosting_type': ['dart', 'gbdt'],
    'num_leaves': (101, 200),
    'max_depth': (8, 12),
    'subsample_for_bin': (21762, 57851),
    'min_split_gain': (0.00299, 0.0997),
    'min_child_weight': (0.00533, 0.0195),
    'min_child_samples': (10, 20),
    'subsample': (0.8077, 0.9976),
    'subsample_freq': (5, 10),
    'colsample_bytree': (0.5112, 0.9971),
    'reg_alpha': (0.00042, 0.9982),
    'reg_lambda': (0.0527, 0.9851)
}



In [141]:
models = [
    #linear_regression,
    #lasso, # changed from lasso_regression
    #ridge, # changed from ridge_regression
    #random_forest, # changed from random_forest_regressor
    #gradient_boosting, # changed from gradient_boosting_regressor
    #adaboost, # changed from adaboost_regressor
    xgboost, # changed from xgboost_regressor
    lgbm
]

param_grids = [
    #lr_params,
    #lasso_params,
    #ridge_params,
    #rf_params,
    #gbr_params,
    #ada_params,
    xgboost_params, # changed from xgb_params
    lgbm_params
]

# Reading data




In [165]:
dfTrain

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,Utilities,LotConfig,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrArea,ExterCond,Foundation,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearsSinceBuilt,YearsSinceRemodel,TotalSurfaceArea,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandSlope_Mod,LandSlope_Sev,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,Fence_GdWo,Fence_MnPrv,Fence_MnWw,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC
0,60,4,65.00,8450,1,0,1,4,5,2,2,0,5,7,5,1,1,13,14,196.00,4,2,2,706.00,6,0.00,150.00,856.00,1,0,1,5,856,854,0,1710,1.00,0.00,2,1,3,1,8,7,0,3,1,2003.00,2.00,548.00,5,5,0,61,0,0,0,0,0,2,2008,9,4,5,5,3483.00,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
1,20,4,80.00,9600,1,0,1,2,24,1,2,0,2,6,8,1,1,8,8,0.00,4,1,0,978.00,6,0.00,284.00,1262.00,1,0,1,5,1262,0,0,1262,0.00,1.00,2,0,3,1,6,7,1,5,1,1976.00,2.00,460.00,5,5,298,0,0,0,0,0,0,5,2007,9,4,31,31,4084.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
2,60,4,68.00,11250,1,0,1,4,5,2,2,0,5,7,5,1,1,13,14,162.00,4,2,2,486.00,6,0.00,434.00,920.00,1,0,1,5,920,866,0,1786,1.00,0.00,2,1,3,1,6,7,1,5,1,2001.00,2.00,608.00,5,5,0,42,0,0,0,0,0,9,2008,9,4,7,6,3668.00,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
3,70,4,60.00,9550,1,0,1,0,6,2,2,0,5,7,5,1,1,14,16,0.00,4,0,0,216.00,6,0.00,540.00,756.00,1,2,1,5,961,756,0,1717,1.00,0.00,1,0,3,1,7,7,1,2,5,1998.00,3.00,642.00,5,5,0,35,272,0,0,0,0,2,2006,9,0,91,36,3536.00,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0
4,60,4,84.00,14260,1,0,1,2,15,2,2,0,5,8,5,1,1,13,14,350.00,4,2,2,655.00,6,0.00,490.00,1145.00,1,0,1,5,1145,1053,0,2198,1.00,0.00,2,1,4,1,9,7,1,5,1,2000.00,3.00,836.00,5,5,192,84,0,0,0,0,0,12,2008,9,4,8,8,4764.00,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,4,62.00,7917,1,0,1,4,8,2,2,0,5,6,5,1,1,13,14,0.00,4,2,6,0.00,6,0.00,953.00,953.00,1,0,1,5,953,694,0,1647,0.00,0.00,2,1,3,1,7,7,1,5,1,1999.00,2.00,460.00,5,5,0,40,0,0,0,0,0,8,2007,9,4,8,7,3593.00,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
1456,20,4,85.00,13175,1,0,1,4,14,2,2,0,2,6,6,1,1,10,11,119.00,4,1,0,790.00,5,163.00,589.00,1542.00,1,4,1,5,2073,0,0,2073,1.00,0.00,2,0,3,1,7,2,2,5,1,1978.00,2.00,500.00,5,5,349,0,0,0,0,0,0,2,2010,9,4,32,22,5506.00,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0
1457,70,4,66.00,9042,1,0,1,4,6,2,2,0,5,7,9,1,1,5,5,0.00,2,4,2,275.00,6,0.00,877.00,1152.00,1,0,1,5,1188,1152,0,2340,0.00,0.00,2,0,4,1,9,7,2,2,1,1941.00,1.00,252.00,5,5,0,60,0,0,0,0,2500,5,2010,9,4,69,4,4704.00,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
1458,20,4,68.00,9717,1,0,1,4,12,2,2,0,2,5,6,3,1,8,8

In [93]:
dfOriginal['SalePrice']

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [ ]:
columns=[
    'YearBuilt','YearRemodAdd'
]

#her


In [171]:
predictions,lp = best_preds_one(models, param_grids,

                                              dfTrain,
                                              dfTest,

                                              dfOriginal['SalePrice'])


[I 2023-08-23 00:47:53,486] A new study created in memory with name: no-name-bf20a5cf-d217-43a6-b687-b5f4beac954e


Evaluating model: XGBRegressor


[I 2023-08-23 00:48:06,290] Trial 0 finished with value: 0.13744479328474662 and parameters: {'n_estimators': 180, 'max_depth': 3, 'learning_rate': 0.3290829581545139, 'gamma': 0, 'reg_lambda': 0.676901805256618, 'reg_alpha': 0.24310494273657687, 'subsample': 0.9484291521559269}. Best is trial 0 with value: 0.13744479328474662.
[I 2023-08-23 00:48:25,137] Trial 1 finished with value: 0.15030218440853343 and parameters: {'n_estimators': 133, 'max_depth': 10, 'learning_rate': 0.36697951824021957, 'gamma': 0, 'reg_lambda': 0.5163729239027234, 'reg_alpha': 0.45136758586937586, 'subsample': 0.8339560603279652}. Best is trial 0 with value: 0.13744479328474662.
[I 2023-08-23 00:48:36,769] Trial 2 finished with value: 0.13727667079244213 and parameters: {'n_estimators': 108, 'max_depth': 7, 'learning_rate': 0.20778425992683458, 'gamma': 0, 'reg_lambda': 0.4114999008696293, 'reg_alpha': 0.7494832784964466, 'subsample': 0.6015801745949299}. Best is trial 2 with value: 0.13727667079244213.
[I 202

Best trial for model XGBRegressor: score 0.12770713494308802, params {'n_estimators': 196, 'max_depth': 5, 'learning_rate': 0.09407472650730998, 'gamma': 0, 'reg_lambda': 0.32899320300031487, 'reg_alpha': 0.188783899404987, 'subsample': 0.7384245208389166}


[I 2023-08-23 00:50:06,529] A new study created in memory with name: no-name-eab03f6c-80a1-4ac9-8d43-cba6e204746a


Model XGBRegressor Metrics:
Mean Absolute Error (MAE): 16225.61176958476
Mean Squared Error (MSE): 813343120.960986
Root Mean Squared Error (RMSE): 28519.171112796845
Root Mean Squared Logarithmic Error (RMSLE): 0.14171425533894072
R² score: 0.8939623819129257
Explained Variance Score: 0.8939623819898854
------------------------------------------------------------------------
Evaluating model: LGBMRegressor


[I 2023-08-23 00:50:17,110] Trial 0 finished with value: 0.14850112394293352 and parameters: {'n_estimators': 240, 'learning_rate': 0.0929036282963506, 'boosting_type': 'rf', 'num_leaves': 85, 'max_depth': 8, 'subsample_for_bin': 299397, 'min_split_gain': 0.962686032066535, 'min_child_weight': 0.002136444883649386, 'min_child_samples': 5, 'subsample': 0.7601660604274367, 'subsample_freq': 5, 'colsample_bytree': 0.5246421185524248, 'reg_alpha': 0.7038131197853031, 'reg_lambda': 0.47959965019204787}. Best is trial 0 with value: 0.14850112394293352.
[I 2023-08-23 00:50:19,560] Trial 1 finished with value: 0.13436323856667537 and parameters: {'n_estimators': 247, 'learning_rate': 0.2699364442064198, 'boosting_type': 'gbdt', 'num_leaves': 121, 'max_depth': 5, 'subsample_for_bin': 227205, 'min_split_gain': 0.013483642809168028, 'min_child_weight': 0.08972635409101297, 'min_child_samples': 19, 'subsample': 0.9895693069843973, 'subsample_freq': 2, 'colsample_bytree': 0.5576530542626794, 'reg_a

KeyboardInterrupt: ignored

In [164]:
create_and_download_submission(predictions[1], filename='submis123123sion.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
from google.colab import files

def create_and_download_submission(predictions, filename='submission.csv'):
    # Read the CSV file that contains the ID of each test sample
    dfid = pd.read_csv(folder_path + 'test.csv')

    # Create a dataframe for the submission
    submission = pd.DataFrame({
        "Id": dfid["Id"],
        "SalePrice": predictions
    })

    # Save the submission dataframe to a CSV file
    submission.to_csv(filename, index=False)

    # Download the CSV file
    files.download(filename)

In [105]:
print(predictions[1].sum()/1459)

178208.13159698423


In [ ]:
for i in range(0,3,1)
  print(predictions[i].sum()/1459)

#swtackig



In [133]:
def best_preds_comb(models, param_grids, X_train_combined, X_test_combined, y, n_trials=15):
    # Split the full training data into train, meta_train and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_train_combined, y, test_size=0.30, random_state=42)
    X_meta_train, X_val, y_meta_train, y_val = train_test_split(
        X_temp, y_temp, test_size=(6/30), random_state=42)

    train_predictions = []
    meta_train_predictions = []
    val_predictions = []
    test_predictions = []
    model_selections = []

    for model, param_grid in zip(models, param_grids):
        pipeline = ModelSelectionPipeline(model, param_grid, n_trials=n_trials)
        pipeline.fit_and_predict(X_train, y_train, X_meta_train)

        train_predictions.append(pipeline.best_model.predict(X_train))
        meta_train_predictions.append(pipeline.best_model.predict(X_meta_train))
        val_predictions.append(pipeline.best_model.predict(X_val))
        test_predictions.append(pipeline.best_model.predict(X_test_combined))
        model_selections.append(pipeline.best_model)


    meta_param_grid  = {
    'n_estimators': (150, 300),
    'learning_rate': (0.1, 0.5), # Narrowing down based on best value
    'boosting_type': ['gbdt', 'dart'], # Removing 'rf' as 'dart' performed best
    'num_leaves': (100, 200), # Narrowing down based on best value
    'max_depth': (8, 12), # Narrowing down based on best value
    'subsample_for_bin': (20000, 60000), # Narrowing down based on best value
    'min_split_gain': (0.0, 0.1), # Narrowing down based on best value
    'min_child_weight': (0.005, 0.02), # Narrowing down based on best value
    'min_child_samples': (10, 20), # Narrowing down based on best value
    'subsample': (0.8, 1.0), # Narrowing down based on best value
    'subsample_freq': (5, 10), # Keeping the same, as varied results
    'colsample_bytree': (0.5, 1.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0)
    }

    meta_model = LGBMRegressor()

    meta_train_predictions_np = np.column_stack(meta_train_predictions)
    test_predictions_np = np.column_stack(test_predictions)

    pipeline = ModelSelectionPipeline(meta_model, meta_param_grid, n_trials=15)
    pipeline.fit_and_predict(meta_train_predictions_np, y_meta_train, test_predictions_np)

    # Calculate the mean squared error of the meta model on the validation data
    val_predictions_np = np.column_stack(val_predictions)
    meta_model_mse = mean_squared_error(y_val, pipeline.best_model.predict(val_predictions_np))
    print(f"Meta model mean squared error: {meta_model_mse}")

    return pipeline.best_predictions, model_selections


In [134]:
predictions,lp = best_preds_comb(models, param_grids,

                                              dfTrain,
                                              dfTest,

                                              dfOriginal['SalePrice'])

[I 2023-08-23 00:22:21,526] A new study created in memory with name: no-name-8dc384fd-16b5-4867-a970-9af43b50a356


Evaluating model: RandomForestRegressor


[I 2023-08-23 00:22:23,362] Trial 0 finished with value: 0.16408289252473357 and parameters: {'n_estimators': 56, 'max_depth': 20, 'min_samples_split': 3, 'min_samples_leaf': 9, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.16408289252473357.
[I 2023-08-23 00:22:26,489] Trial 1 finished with value: 0.14850120252886606 and parameters: {'n_estimators': 82, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.14850120252886606.
[I 2023-08-23 00:22:27,557] Trial 2 finished with value: 0.16140637573245628 and parameters: {'n_estimators': 33, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.14850120252886606.
[I 2023-08-23 00:22:28,683] Trial 3 finished with value: 0.15233651145701724 and parameters: {'n_estimators': 49, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.148501202528

Best trial for model RandomForestRegressor: score 0.14850120252886606, params {'n_estimators': 82, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}


[I 2023-08-23 00:22:51,870] A new study created in memory with name: no-name-590a0f7d-ba2e-4e94-b684-82e50f5f3f73


...

Best trial for model LGBMRegressor: score 0.1265389568869303, params {'n_estimators': 263, 'learning_rate': 0.1753740671312475, 'boosting_type': 'dart', 'num_leaves': 199, 'max_depth': 5, 'subsample_for_bin': 164530, 'min_split_gain': 0.04874547425632887, 'min_child_weight': 0.0038965285569840923, 'min_child_samples': 15, 'subsample': 0.7036141814329748, 'subsample_freq': 10, 'colsample_bytree': 0.5115602490882938, 'reg_alpha': 0.2462901953900869, 'reg_lambda': 0.29081818213880745}
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001129 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3295
[LightGBM] [Info] Number of data points in the train set: 1022, number of used features: 91
[LightGBM] [Info] Start training from score 181312.692759
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2023-08-23 00:26:16,521] A new study created in memory with name: no-name-0e3fe536-b29b-4e34-b54b-e4d8e2c1604e



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Evaluating model: LGBMRegressor


[I 2023-08-23 00:26:17,987] Trial 0 finished with value: 0.14554717920912968 and parameters: {'n_estimators': 231, 'learning_rate': 0.4870657790681997, 'boosting_type': 'dart', 'num_leaves': 171, 'max_depth': 10, 'subsample_for_bin': 49757, 'min_split_gain': 0.09637828725134491, 'min_child_weight': 0.015975323962139187, 'min_child_samples': 15, 'subsample': 0.9874782537546754, 'subsample_freq': 10, 'colsample_bytree': 0.8114480803440336, 'reg_alpha': 0.047683880638901166, 'reg_lambda': 0.12458462324499098}. Best is trial 0 with value: 0.14554717920912968.
[I 2023-08-23 00:26:18,827] Trial 1 finished with value: 0.1467517249830686 and parameters: {'n_estimators': 259, 'learning_rate': 0.1310154477830451, 'boosting_type': 'gbdt', 'num_leaves': 145, 'max_depth': 8, 'subsample_for_bin': 20166, 'min_split_gain': 0.04118531545713483, 'min_child_weight': 0.018486243981260943, 'min_child_samples': 10, 'subsample': 0.933026924385451, 'subsample_freq': 10, 'colsample_bytree': 0.7646126878174567,

Best trial for model LGBMRegressor: score 0.1403002404384696, params {'n_estimators': 196, 'learning_rate': 0.23428267798434993, 'boosting_type': 'dart', 'num_leaves': 127, 'max_depth': 8, 'subsample_for_bin': 59018, 'min_split_gain': 0.06860283346278098, 'min_child_weight': 0.011737768536714705, 'min_child_samples': 12, 'subsample': 0.8563171743613388, 'subsample_freq': 7, 'colsample_bytree': 0.9549642500086614, 'reg_alpha': 0.9887033612801862, 'reg_lambda': 0.6153154025811349}
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000060 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 351
[LightGBM] [Info] Number of data points in the train set: 350, number of used features: 3
[LightGBM] [Info] Start training from score 183006.234286
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]